In [10]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from pandas.io.json import json_normalize
from census import Census

# Census API Key
from config import api_key

# Identify the data we are interested in
desired_labels = "NAME,B19013_001E,B19301_001E,B17001_002E,B17012_002E,B01003_001E,B25077_001E"

# Median Household Income
# "B19013_001E"
# Median household income in the past 12 months

# Income Per Capita
# "B19301_001E"
# Per capita income in the past 12 months

# Poverty Count
# "B17001_002E"
# Number of persons whose income in the past 12 months is below the poverty level

# Family Poverty Count
# "B17012_002E"
# Number of families below the poverty level in the past 12 months

# Population
# "B01003_001E"
# Total population

# Median Home Value
# "B25077_001E"
# average value of an owner-occupied home

In [11]:
# Import the zipcodes of Sacramento County
zip_csv = "./Resources/ZipCode.csv"
zip_df = pd.read_csv(zip_csv)
zip_codes_sac = zip_df["ZipCode"].astype(str)


In [12]:
# Request the ACS5 2014 and 2015 data from Census Bureau's API
# Using requests.json instead of Census library because the endpoint may have been different from 2016 and beyond

# Make an empty dictionary to hold the dfs
census_output = {}

# Create lists to hold data values
zipcodes = []
population = []
poverty_count = []
f_poverty_count = []
income = []
pc_income = []
home_value = []
year = []

# Run for loops to get response from API
for years, suffixx in zip([2014, 2015], ["a", "b"]):
    base_url = f"https://api.census.gov/data/{years}/acs/acs5?get={desired_labels}&for=zip%20code%20tabulation%20area:*&key={api_key}"
    response = requests.get(base_url).json()

    first = False
    for item in response:
        if first is False:
            first = True
            continue
        
        # Append the response into lists created and put them together into data frames
        year.append(years)
        zipcodes.append(item[7])
        income.append(item[1])
        pc_income.append(item[2])
        poverty_count.append(item[3])
        f_poverty_count.append(item[4])
        population.append(item[5])
        home_value.append(item[6])

    census1415 = {
        "ZipCode": zipcodes,
        "Population": population,
        "Poverty Count": poverty_count,
        "Family Poverty Count": f_poverty_count,
        "Median Household Income": income,
        "Per Capita Income": pc_income,
        "Median Home Value": home_value
                }

    census_pd_1415 = pd.DataFrame(data= census1415)

    census_pd_1415["Year"] = years
    census_pd_1415["Poverty Rate"] = (100 * census_pd_1415["Poverty Count"].astype(float) \
    / census_pd_1415["Population"].astype(float)).round(2)

    census_pd_1415 = census_pd_1415[[
        "Year",
        "ZipCode", 
        "Population", 
        "Poverty Count", 
        "Poverty Rate", 
        "Family Poverty Count", 
        "Median Household Income", 
        "Per Capita Income", 
        "Median Home Value"]]

# Filter ZipCode: limited to Sac County
    census_output[suffixx] = census_pd_1415[census_pd_1415["ZipCode"].isin(zip_codes_sac)]

# Reset the lists for 2015
    zipcodes = []
    population = []
    poverty_count = []
    f_poverty_count = []
    income = []
    pc_income = []
    home_value = []
    year = []



In [13]:
# Request the ACS5 2016, 2017, and 2018 data from Census Bureau's API
# Using the Census library.

# Make a for loop
for data_year, suffix in zip([2016, 2017, 2018], ["c", "d", "e"]):
    census_data = Census(api_key, year= data_year).acs5.get(("NAME", "B19013_001E", "B19301_001E", "B17001_002E",
                          "B17012_002E", "B01003_001E", "B25077_001E"), {'for': 'zip code tabulation area:*'})
# Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

# Column Reordering
    census_pd = census_pd.rename(columns={
        "B01003_001E": "Population", 
        "B17001_002E": "Poverty Count", 
        "B17012_002E": "Family Poverty Count",
        "B19013_001E": "Median Household Income", 
        "B19301_001E": "Per Capita Income", 
        "B25077_001E": "Median Home Value",
        "NAME": "Name", "zip code tabulation area": "ZipCode"})

    census_pd["Year"] = data_year

# Add in Poverty Rate (Poverty Count / Population)
    census_pd["Poverty Rate"] = (100 * census_pd["Poverty Count"].astype(float) / census_pd["Population"].astype(float)).round(2)

# Final DataFrame
    census_pd = census_pd[[
        "Year",
        "ZipCode", 
        "Population", 
        "Poverty Count", 
        "Poverty Rate", 
        "Family Poverty Count", 
        "Median Household Income", 
        "Per Capita Income", 
        "Median Home Value"]]

# Filter ZipCode: limited to Sac County
    census_sac = census_pd[census_pd["ZipCode"].isin(zip_codes_sac)]
    census_output[suffix] = census_sac


In [14]:
# Combining the data frames of each year
census_output_final = pd.concat([census_output["a"], census_output["b"], census_output["c"], census_output["d"], census_output["e"]])


In [15]:
# I want to make a for loop to unify the data types for labels

for labels in census_output_final.keys():
    if labels == "Year":
        continue
    elif labels == "ZipCode":
        continue
    else:
        census_output_final[labels] = census_output_final[labels].astype(float)

In [16]:
# Drop bad data (95680, 95652)
census_output_final = census_output_final.dropna(how= "any")
census_output_final = census_output_final.loc[(census_output_final["Median Home Value"] > -666666666)]
census_output_final = census_output_final.loc[(census_output_final["Median Household Income"] > -666666666)]

# Align the ZipCodes across years, which means we need to drop 95615
census_output_final = census_output_final.loc[(census_output_final["ZipCode"] != "95615")]

# Attempts to drop bad data using a for loop
# bad_vals = [-666666666.0]
# for bad_val in bad_vals:
#     census_output_final.drop(census_output_final[census_output_final["Median Home Value"] == bad_val].index)

In [17]:
# Display the final df
census_output_final

,Year,ZipCode,Population,Poverty Count,Poverty Rate,Family Poverty Count,Median Household Income,Per Capita Income,Median Home Value
30966,2014,95608,60729.0,8301.0,13.67,1476.0,55256.0,34606.0,304200.0
30967,2014,95610,43572.0,7174.0,16.46,1136.0,50536.0,25288.0,217100.0
30975,2014,95621,41474.0,5842.0,14.09,1069.0,51827.0,24379.0,173700.0
30977,2014,95624,63131.0,6848.0,10.85,1266.0,80233.0,29752.0,270400.0
30979,2014,95626,6335.0,792.0,12.50,182.0,60579.0,22576.0,193900.0
...,...,...,...,...,...,...,...,...,...
31255,2018,95838,39187.0,9989.0,25.49,1847.0,44849.0,17299.0,201900.0
31256,2018,95841,20586.0,3964.0,19.26,885.0,47215.0,25167.0,233300.0
31257,2018,95842,32269.0,7050.0,21.85,1386.0,50412.0,22419.0,222300.0
31258,2018,95843,46709.0,5818.0,12.46,1248.0,78102.0,27488.0,299900.0


In [18]:
# Export final df to csv file
census_output_final.to_csv("./Resources/census_output_final.csv", encoding="utf-8", index=False)